#Cats vs. Dogs Kernel

Welcome to my Kernel on the Cats vs Dogs competition. This is my first time working with image data so you will see various links that aided me in learning and completing this Kernel. The goal here is to use a Convolutional Neural Network (CNN) to predict if an image is a picture of a Cat or Dog and then evaluate how well the model did by using a validation set.

In [ ]:
import numpy as np 
import pandas as pd 
import cv2
import os
from random import shuffle
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
import matplotlib.pyplot as plt

#The following code allows us to refer to the directories where the images are located.
#This will help later when it comes time to load in the images and shape them.
Train_dir = '../input/train'
Test_dir = '../input/test'
IMG_SIZE = 100

#Code that helped with loading in images and processing images can be found here https://www.youtube.com/watch?v=gT4F3HGYXf4&t=554s

This was my first time working with true image data so the code for processing the image data in this Kernel is modeled after the code in the video below. Since this is my first time loading in image data in this matter, I will try to describe the code line by line.

Link: https://www.youtube.com/watch?v=gT4F3HGYXf4

In [ ]:
#The code below aids in labeling each image as "cat" or "dog"
#The fuction below also labels the images as one-hot arrays
#so that they can be passed through the CNN as output labels.
def label_img(img):
    word_label = img.split('.')[0]
    if word_label == 'cat':
        return [1,0]
    elif word_label == 'dog':
        return [0,1]

In [ ]:
#The second line of the code above was confusing to me at first so I decided to
#make a print statement that helped me to understand it better.
#Running the print statement below shows that it grabs the text 
#that indicates if the picture is a cat or dog. It splits the image by '.'
#and then takes the 0-ith object in the index which is "cat" or "dog".
words = 'cat.0.jpg'
words2 = words.split('.')[0]
print(words2)

The code below is the function that will create our training data. Some notes will be included in the code to help break it down.

In [ ]:
def create_train_data():
    training_data = []
    #os.listdir returns a list of all of the files and folders in the path
    #So we pass the path to the training directory that we created earlier
    #https://www.youtube.com/watch?v=iI2zR1WLPZ8
    for img in os.listdir(Train_dir):
        label = label_img(img)
        #os.path.join seems to create a path to each individual image by
        #combining the path to the training directory and the image name
        #https://docs.python.org/3/library/os.path.html#os.path.join
        path = os.path.join(Train_dir,img)
        #The code below uses cv2 to read in the image in Grayscale and then resize
        #the images to 100x100 which we created earlier with the variable IMG_SIZE
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        #Appends the result of each image to a numpy array so we can work with it later
        training_data.append([np.array(img), np.array(label)])
        shuffle(training_data)
    return training_data


In [ ]:
#Run function to get our training data
training_data = create_train_data()

The code below creates X and Y arrays to be passed into the model. X is reshaped to 100x100 and divided by 255 as a normalization technique. The model is then created and compiled; for more on the normalization techniques used and creating/ compiling CNNs I wrote an introduction to CNNs here: https://github.com/kasmith11/Intro-to-Convolution-Neural-Networks

In [ ]:
X = np.array([i[0] for i in training_data])
X = X.reshape(-1,100,100,1)
X = X/255
Y = np.array([i[1] for i in training_data])

print(X.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size = (3,3), input_shape=(100,100,1), activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy', 'mae'])

After creating and compiling the model, it is time to fit the data to the training set. We will use 30% of the training data as the validation set.

In [ ]:
Fit = model.fit(X, Y, epochs = 5, validation_split = 0.30)

The plots below look at the models accuracy and loss overtime. Based on the graphs below, it seems that the model is overfitting the training data. This can be seen by the model preforming significantly worse on "unseen" validation samples. To improve this model in the future we can use things such as Dropout and other methods that help with over fitting. The links below helped with plotting and calling the accuracy and loss of the model.

https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

https://keras.io/callbacks/

In [ ]:
plt.plot(Fit.history['loss'])
plt.plot(Fit.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.show()

In [ ]:
plt.plot(Fit.history['acc'])
plt.plot(Fit.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.show()